In [1]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from itertools import chain,combinations

# загрузка данных из файла
data = pd.read_excel('full_data.xlsx')


In [3]:
df = pd.read_csv('pricis_befori_may.txt',delimiter=',')
df.head()

,Country,ISO3 Code,Date,Price (EUR/MWhe)
0,Austria,AUT,2015-01-01,31.68
1,Austria,AUT,2015-02-01,36.69
2,Austria,AUT,2015-03-01,31.30
3,Austria,AUT,2015-04-01,29.78
4,Austria,AUT,2015-05-01,25.33


In [5]:
df.groupby('Date')['Price (EUR/MWhe)'].mean()/1000

Date
2015-01-01    0.039409
2015-02-01    0.041030
2015-03-01    0.036714
2015-04-01    0.035725
2015-05-01    0.032623
                ...   
2022-12-01    0.242479
2023-01-01    0.125556
2023-02-01    0.132162
2023-03-01    0.103551
2023-04-01    0.095798
Name: Price (EUR/MWhe), Length: 100, dtype: float64

In [ ]:
# выделение признаков и целевой переменной
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values

# масштабирование признаков
scaler = StandardScaler()
X = scaler.fit_transform(X)

# разбиение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

# оценка качества модели на тестовых данных
score = model.score(X_test, y_test)
print(f'R^2 score: {score:.2f}')

In [18]:
def overtest(model,X,y):
    # задаем количество фолдов
    n_splits = 5
    kf = KFold(n_splits=n_splits)

    # создаем пустой список для сохранения ошибок на обучающих и тестовых данных
    train_errors, test_errors = [], []

    # итерируемся по фолдам
    for train_index, test_index in kf.split(X):
        # разделяем данные на обучающие и тестовые
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]

        # масштабируем данные
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # обучаем модель
        model.fit(X_train_scaled, y_train)

        # сохраняем ошибки на обучающих и тестовых данных
        train_errors.append(mean_squared_error(y_train, model.predict(X_train_scaled)))
        test_errors.append(mean_squared_error(y_test, model.predict(X_test_scaled)))

    # выводим среднюю ошибку на обучающих и тестовых данных
    print(model)
    print('Mean train error:', np.mean(train_errors))
    print('Mean test error:', np.mean(test_errors))
    print('Koef:',np.mean(test_errors)/np.mean(train_errors))


In [ ]:
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = ridge.predict(X_test)

# Оценка качества модели на тестовой выборке
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f'RMSE на тестовой выборке с регуляризацией: {rmse}')

# Создание базовой модели без регуляризации
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred_base = lin_reg.predict(X_test)

# Оценка качества базовой модели на тестовой выборке
mse_base = mean_squared_error(y_test, y_pred_base)
rmse_base = mse_base ** 0.5
print(f'RMSE на тестовой выборке без регуляризации: {rmse_base}')

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# создание модели Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# обучение модели
rf_model.fit(X_train, y_train)

# оценка качества на тестовой выборке
rf_rmse = mean_squared_error(y_test, rf_model.predict(X_test), squared=False)
print("RMSE для Random Forest: ", rf_rmse)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# создание модели Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)

# обучение модели
gb_model.fit(X_train, y_train)

# оценка качества на тестовой выборке
gb_rmse = mean_squared_error(y_test, gb_model.predict(X_test), squared=False)
print("RMSE для Gradient Boosting: ", gb_rmse)


In [ ]:
from sklearn.ensemble import BaggingRegressor

# создание модели Bagging
bg_model = BaggingRegressor(n_estimators=100, random_state=42)

# обучение модели
bg_model.fit(X_train, y_train)

# оценка качества на тестовой выборке
bg_rmse = mean_squared_error(y_test, bg_model.predict(X_test), squared=False)
print("RMSE для Bagging: ", bg_rmse)


In [ ]:
bg_model.score(X_test,y_test)

In [ ]:
overtest(bg_model)

In [ ]:
0.008321299884767874/0.00015618095850719686

In [ ]:
overtest(gb_model)

In [ ]:
overtest(rf_model)

In [4]:
t_col = ['Geothermal',
         'Solar',
         'Wind', 
         #'Other Renewables',
         'Hydro',
         'Combustible Renewables',
         #'Coal, Peat and Manufactured Gases',
         #'Oil and Petroleum Products',
         'Natural Gas',
         #'Other Combustible Non-Renewables',
         'UNMPR'
        ]

In [5]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

stuff = [1, 2, 3]
lst = [combo for i, combo in enumerate(powerset(t_col), 1)]

In [ ]:
lst = [sorted(x) for x in lst if x]
lst

In [19]:
def check_all(data,cols):
    X = data[cols]
    y = data['Price(EUR/GWh)']

    # масштабирование признаков
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # разбиение данных на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # обучение модели
    model = RandomForestRegressor(n_estimators=100,max_features='sqrt',max_depth=5,min_samples_leaf=1,min_samples_split=2)
    model.fit(X_train, y_train)

    # оценка качества модели на тестовых данных
    score = model.score(X_test, y_test)
    if abs(score) > 0.6:
        print(cols)
        print(f'R^2 score: {score:.2f}')
        overtest(model,X,y)

In [20]:
for cols in lst:
    check_all(data,cols)
    print('-'*10)

----------
----------
----------
----------
----------
----------
['UNMPR']
R^2 score: 0.81
RandomForestRegressor(max_depth=5, max_features='sqrt')
Mean train error: 0.0011963730304754957
Mean test error: 0.008807656768535179
Koef: 7.361965327013929
----------
----------
----------
----------
----------
----------
['Geothermal', 'UNMPR']
R^2 score: 0.88
RandomForestRegressor(max_depth=5, max_features='sqrt')
Mean train error: 0.0003787549322583026
Mean test error: 0.008664909345053307
Koef: 22.877350516307015
----------
----------
----------
----------
----------
['Solar', 'UNMPR']
R^2 score: 0.81
RandomForestRegressor(max_depth=5, max_features='sqrt')
Mean train error: 0.00042673976039626324
Mean test error: 0.008031721918020037
Koef: 18.821123934085534
----------
----------
----------
----------
['UNMPR', 'Wind']
R^2 score: 0.89
RandomForestRegressor(max_depth=5, max_features='sqrt')
Mean train error: 0.00037333199114079695
Mean test error: 0.008674643886924693
Koef: 23.2357368047068